In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install --upgrade openpyxl

     |████████████████████████████████| 3.8 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 38.9 MB/s 
     |████████████████████████████████| 596 kB 40.5 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 6.5 MB 31.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

In [ ]:
import scipy
import pandas as pd
import numpy as np
from NER.model import predict
from tqdm import tqdm, trange

In [ ]:
data = pd.read_excel('/content/sample_data/G34 для разметки.xlsx')

In [ ]:
all_results = pd.DataFrame(['', '', '', '']).T
all_results = all_results.head(0)
all_results.columns = ('word', 'label', 'confidence', 'code')

In [ ]:
data = data[pd.notna(data['Полное наименование ru'])].head(1000)

In [ ]:
data_list = list(data['Полное наименование ru'].apply(lambda p: p.lower()))

In [ ]:
code_list = list(data['Код'])

In [ ]:
a = [(predict.make_batch_prediction(data_list[i:i+1]),  code_list[i:i+1])for i in range(0, len(data_list), 1)]

In [ ]:
a_word_all = []
a_label_all = []
a_conf_all = []
a_values_all = []
a_step_all = []

In [ ]:
for index in range(1):
    b = [(a_word_all.extend(i[0][index][0]), a_values_all.extend(i[0][index][3]), a_label_all.extend(i[0][index][1]), 
                a_conf_all.extend(i[0][index][2]), a_step_all.extend([i[1][0]]*len(i[0][index][0]))) for step, i in enumerate(a)]

In [ ]:
results = pd.DataFrame([a_step_all, a_word_all, a_values_all, a_label_all, a_conf_all]).T
results.columns = ('code', 'word', 'values', 'label', 'confidence')
results

,code,word,values,label,confidence
0,1022290,кольцо,None,Вид изделия,0.994791
1,1022290,фрикционное,None,особенности конструкции,0.999728
2,1022290,sbf-38,None,маркировка,0.993255
3,1022290,; код 930000-38; чертеж 302908 поз.3;,None,,0
4,1022290,материалsealide,None,производитель/наименование,0.965086
...,...,...,...,...,...
14211,1102194,.190;,None,,0
14212,1102194,для компрессора,None,особенности конструкции,0.618211
14213,1102194,вшв,None,маркировка,0.971457
14214,1102194,- 3/100; зав.№41049; 2004г; оао,None,,0


In [ ]:
results = pd.merge(results, data[['Код', 'Полное наименование ru']], left_on = ['code'], 
         right_on=['Код'])[['code', 'word', 'values', 'label', 'confidence', 'Полное наименование ru']]

In [ ]:
results

,code,word,values,label,confidence,Полное наименование ru
0,1022290,кольцо,None,Вид изделия,0.994791,Кольцо фрикционное SBF-38; код 930000-38; черт...
1,1022290,фрикционное,None,особенности конструкции,0.999728,Кольцо фрикционное SBF-38; код 930000-38; черт...
2,1022290,sbf-38,None,маркировка,0.993255,Кольцо фрикционное SBF-38; код 930000-38; черт...
3,1022290,; код 930000-38; чертеж 302908 поз.3;,None,,0,Кольцо фрикционное SBF-38; код 930000-38; черт...
4,1022290,материалsealide,None,производитель/наименование,0.965086,Кольцо фрикционное SBF-38; код 930000-38; черт...
...,...,...,...,...,...,...
14211,1102194,.190;,None,,0,Клапан предохранительный 2 ступени 391.314.02....
14212,1102194,для компрессора,None,особенности конструкции,0.618211,Клапан предохранительный 2 ступени 391.314.02....
14213,1102194,вшв,None,маркировка,0.971457,Клапан предохранительный 2 ступени 391.314.02....
14214,1102194,- 3/100; зав.№41049; 2004г; оао,None,,0,Клапан предохранительный 2 ступени 391.314.02....


In [ ]:
(results.head(1000)).to_excel('/content/g34_results.xlsx')

In [ ]:
res_g = results[(results['label'] == 'маркировка') & (results['confidence']>0.7)].groupby(by=['word', 'Полное наименование ru'], as_index=False).max()
res_g

,word,Полное наименование ru,code,values,label,confidence
0,-1x34p,ЦЕПЬ К ВЕЗДЕХОДУ ARGO8X8 AVENGER -CHAIN IDLER ...,1097466,NaN,маркировка,0.880170
1,-4с,Шланг от помпы к рубашке приемного коллектора ...,1150849,NaN,маркировка,0.868933
2,10 4sp,Рукав высокого давления РВД 371LT-10 4SP DN 16...,1132654,NaN,маркировка,0.839137
3,120041s,Клапан впускной 5149014 для ДВС автокрана NK-1...,1075302,NaN,маркировка,0.755896
4,120041s,Клапан ускорительный 533-66042000 для автокран...,1075246,NaN,маркировка,0.706582
...,...,...,...,...,...,...
147,ямз-238,Двигатель внутреннего сгорания ЯМЗ-238 М2-6 №1...,1132686,NaN,маркировка,0.991560
148,ямз-238,Двигатель внутреннего сгорания ЯМЗ-238 №10115825,1132684,NaN,маркировка,0.970900
149,ямз-238,Двигатель внутреннего сгорания ЯМЗ-238 №10119323,1132689,NaN,маркировка,0.980336
150,ямз-238,Двигатель внутреннего сгорания ЯМЗ-238 №26315,1132691,NaN,маркировка,0.964724


In [ ]:
res_g2 = results[(results['label'] == 'маркировка') & (results['confidence']>0.7)].groupby(by=['word'], as_index=False).count()
res_g2 = res_g2[res_g2['code']>=5].sort_values(by=['code'], ascending=False)
res_g2

,word,code,values,label,confidence,Полное наименование ru
76,ст,22,0,22,22,22
20,lr-632b,5,0,5,5,5
25,vf,5,0,5,5,5


In [ ]:
pd.merge(res_g2, res_g, on=['word']).groupby(by=['word'], 
                                as_index=False).max().sort_values(by=['code_x'], ascending=False)[['word', 
                                'Полное наименование ru_y']].to_excel('/content/drive/MyDrive/distBert/Colab Notebooks/data/g39_marks.xlsx', index=None)